In [ ]:
!pip install mediapipe
import mediapipe as mp 
import cv2


In [ ]:
!pip install numpy
import numpy as np

In [ ]:
HSV = [np.array([136, 87, 111], np.uint8), np.array([180, 255, 255], np.uint8),
       np.array([25, 52, 72], np.uint8)  , np.array([76, 255, 255],  np.uint8),
       np.array([94, 80, 2], np.uint8)   , np.array([150, 255, 255], np.uint8)]

In [ ]:
mpDraw = mp.solutions.drawing_utils
mpFaceMesh = mp.solutions.face_mesh
faceMesh = mpFaceMesh.FaceMesh(max_num_faces=2)
drawSpec = mpDraw.DrawingSpec(thickness=1, circle_radius=2)

In [ ]:
def prespective(biggest,img):
    scale = 1
    pts1 = np.float32(biggest)  # PREPARE POINTS FOR WARP
    highttrace = (biggest[1][1]-biggest[0][1])*scale
    widthtrace = (biggest[2][0]-biggest[0][0])*scale
    pts2 = np.float32([[0, 0],  [0, highttrace], [widthtrace, highttrace], [widthtrace, 0]])  # PREPARE POINTS FOR WARP
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarpColored = cv2.warpPerspective(img, matrix, (widthtrace, highttrace))
    return imgWarpColored

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import numpy as np

def init_camera():
  """Create objects and functions in HTML/JavaScript to access local web camera"""

  js = Javascript('''

    // global variables to use in both functions
    var div = null;
    var video = null;   // <video> to display stream from local webcam
    var stream = null;  // stream from local webcam
    var canvas = null;  // <canvas> for single frame from <video> and convert frame to JPG
    var img = null;     // <img> to display JPG after processing with `cv2`

    async function initCamera() {
      // place for video (and eventually buttons)
      div = document.createElement('div');
      document.body.appendChild(div);

      // <video> to display video
      video = document.createElement('video');
      video.style.display = 'block';
      div.appendChild(video);

      // get webcam stream and assing to <video>
      stream = await navigator.mediaDevices.getUserMedia({video: true});
      video.srcObject = stream;

      // start playing stream from webcam in <video>
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // <canvas> for frame from <video>
      canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      //div.appendChild(input_canvas); // there is no need to display to get image (but you can display it for test)

      // <img> for image after processing with `cv2`
      img = document.createElement('img');
      img.width = video.videoWidth;
      img.height = video.videoHeight;
      div.appendChild(img);
    }

    async function takeImage(quality) {
      // draw frame from <video> on <canvas>
      canvas.getContext('2d').drawImage(video, 0, 0);

      // stop webcam stream
      //stream.getVideoTracks()[0].stop();

      // get data from <canvas> as JPG image decoded base64 and with header "data:image/jpg;base64,"
      return canvas.toDataURL('image/jpeg', quality);
      //return canvas.toDataURL('image/png', quality);
    }

    async function showImage(image) {
      // it needs string "data:image/jpg;base64,JPG-DATA-ENCODED-BASE64"
      // it will replace previous image in `<img src="">`
      img.src = image;
      // TODO: create <img> if doesn't exists, 
      // TODO: use `id` to use different `<img>` for different image - like `name` in `cv2.imshow(name, image)`
    }

  ''')

  display(js)
  eval_js('initCamera()')

def take_frame(quality=0.8):
  """Get frame from web camera"""

  data = eval_js('takeImage({})'.format(quality))  # run JavaScript code to get image (JPG as string base64) from <canvas>

  header, data = data.split(',')  # split header ("data:image/jpg;base64,") and base64 data (JPG)
  data = b64decode(data)  # decode base64
  data = np.frombuffer(data, dtype=np.uint8)  # create numpy array with JPG data

  img = cv2.imdecode(data, cv2.IMREAD_UNCHANGED)  # uncompress JPG data to array of pixels

  return img

def show_frame(img, quality=0.8):
  """Put frame as <img src="data:image/jpg;base64,...."> """

  ret, data = cv2.imencode('.jpg', img)  # compress array of pixels to JPG data

  data = b64encode(data)  # encode base64
  data = data.decode()  # convert bytes to string
  data = 'data:image/jpg;base64,' + data  # join header ("data:image/jpg;base64,") and base64 data (JPG)

  eval_js('showImage("{}")'.format(data))  # run JavaScript code to put image (JPG as string base64) in <img>

In [ ]:
import cv2
import os
from google.colab.patches import cv2_imshow

# init JavaScript code
init_camera()

while True:
    try:
        img = take_frame()

        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        results = faceMesh.process(imgRGB)
        id067=id203=(0,0)
        maxma = 0
        mask = []
        if results.multi_face_landmarks:
            for faceLms in results.multi_face_landmarks:
                # mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS,drawSpec, drawSpec)
                for id,lm in enumerate(faceLms.landmark):
                    #print(lm)
                    ih, iw, ic = img.shape
                    x, y = int(lm.x * iw), int(lm.y * ih)
                    # cv2.putText(img,str(id),(x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0, 255))
                    if(id == 67):
                        id067 = (x,y)
                    if(id == 293):
                        id203 = (x,y)
        if id067[1] > id203[1]:
            pt1=id067
            pt2=id203
            id067 = (pt1[0],pt2[1])
            id203 = (pt2[0],pt1[1])

        array = [[id067[0],id067[1]], [id067[0], id203[1]], [id203[0], id203[1]], [id203[0], id067[1]]]
        imageFrame = prespective(array,img)

        hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)
        kernal = np.ones((5, 5), "uint8")
        for i in range(3):
            mask.append(cv2.inRange(hsvFrame, HSV[(i*2)], HSV[(i*2) + 1]))
            m = cv2.dilate(mask[i], kernal)
            red = cv2.bitwise_and(imageFrame, imageFrame, mask = m)
            contours, hierarchy = cv2.findContours(m,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
            for pic, contour in enumerate(contours):
                area = cv2.contourArea(contour)
                if (area > 300):
                    x, y, w, h = cv2.boundingRect(contour)
                    imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0, 0, 255), 2)
                    cv2.putText(img, "Face Shield detected", (id067[0] + x, id067[1] + y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0))
                    maxma = 1
                    print("Face Shield detected")
        if(maxma==0):
            cv2.putText(img, "No shield detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0, 255))
            print("No shield detected")
        
        cv2_imshow(img)  # it creates new image for every frame (it doesn't replace previous image) so it is useless
        show_frame(img)  # it replace previous image
        
    except Exception as err:
        print('Exception:', err)

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      // Wait for Capture to be clicked.
      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
# from google.colab import files
# files.upload()

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  
  # Show the image which was just taken.
  display(Image(filename))
except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
try:

  img = cv2.imread('/content/photo.jpg')
  imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
  results = faceMesh.process(imgRGB)
  id067=id203=(0,0)
  maxma = 0
  mask = []
  if results.multi_face_landmarks:
      for faceLms in results.multi_face_landmarks:
          # mpDraw.draw_landmarks(img, faceLms, mpFaceMesh.FACE_CONNECTIONS,drawSpec, drawSpec)
          for id,lm in enumerate(faceLms.landmark):
              #print(lm)
              ih, iw, ic = img.shape
              x, y = int(lm.x * iw), int(lm.y * ih)
              # cv2.putText(img,str(id),(x,y),cv2.FONT_HERSHEY_SIMPLEX, 0.3, (0,0, 255))
              if(id == 67):
                  id067 = (x,y)
              if(id == 293):
                  id203 = (x,y)
  if id067[1] > id203[1]:
      pt1=id067
      pt2=id203
      id067 = (pt1[0],pt2[1])
      id203 = (pt2[0],pt1[1])

  array = [[id067[0],id067[1]], [id067[0], id203[1]], [id203[0], id203[1]], [id203[0], id067[1]]]
  imageFrame = prespective(array,img)

  hsvFrame = cv2.cvtColor(imageFrame, cv2.COLOR_BGR2HSV)
  kernal = np.ones((5, 5), "uint8")
  for i in range(3):
      mask.append(cv2.inRange(hsvFrame, HSV[(i*2)], HSV[(i*2) + 1]))
      m = cv2.dilate(mask[i], kernal)
      red = cv2.bitwise_and(imageFrame, imageFrame, mask = m)
      contours, hierarchy = cv2.findContours(m,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
      for pic, contour in enumerate(contours):
          area = cv2.contourArea(contour)
          if (area > 300):
              x, y, w, h = cv2.boundingRect(contour)
              imageFrame = cv2.rectangle(imageFrame, (x, y),(x + w, y + h),(0, 0, 255), 2)
              cv2.putText(img, "Face Shield detected", (id067[0] + x, id067[1] + y), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0))
              maxma = 1
              print("Face Shield detected")
  if(maxma==0):
      cv2.putText(img, "No shield detected", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0,0, 255))
      print(" No Face Shield detected")
  cv2_imshow(img)

except Exception as err:
  # Errors will be thrown if the user does not have a webcam or if they do not
  # grant the page permission to access it.
  print(str(err))
  